# DeepLinc

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 05.01.2023
- **Date of Last Modification:** 22.07.2024

- The DeepLinc source code is available at https://github.com/xryanglab/DeepLinc.
- The corresponding publication is "Li, R. & Yang, X. De novo reconstruction of cell interaction landscapes from single-cell spatial transcriptome data with DeepLinc. Genome Biol. 23, 124 (2022)".
- The logic to run DeepLinc is encapsulated into the Python script ```../DeepLinc/deeplinc.py```.
- We train DeepLinc models on the benchmarking datasets with different adjacency matrices corresponding to 4, 8, 12, 16, and 20 average neighbors.
- We compute 2 runs per number of average neighbors resulting in a total of 10 runs. Each of the 2 runs uses a different random seed (seed 0 and seed 1).
- To train a DeepLinc model with 4 average neighbors, open a terminal, navigate to the ```../DeepLinc``` folder, and run ```python deeplinc.py -e ../../../datasets/st_data/gold/seqfish_mouse_organogenesis_embryo2_counts.csv -a ../../../datasets/st_data/gold/seqfish_mouse_organogenesis_embryo2_adj4.csv -c ../../../datasets/st_data/gold/seqfish_mouse_organogenesis_embryo2_coords.csv -r ../../../datasets/st_data/gold/seqfish_mouse_organogenesis_embryo2_cell_types.csv -n run1 -i 40 --seed 0``` (after the data has been prepared further down in this notebook).
- The results of the trained DeepLinc models (latent space features & reconstructed adjacency matrices) are manually stored in ```../../../datasets/benchmark_data/deeplinc/{dataset}```.

Run the deeplinc script and this notebook in the deeplinc environment, installable from ```('../../../envs/environment_deeplinc.yaml')```.

## 1. Setup

### 1.1 Import Libraries

In [ ]:
import csv
import json
import os
import re
from datetime import datetime

import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
import squidpy as sq

### 1.2 Define Parameters

In [ ]:
model_name = "deeplinc"
latent_key = f"{model_name}_latent"

### 1.3 Run Notebook Setup

In [ ]:
sc.set_figure_params(figsize=(6, 6))

In [ ]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [ ]:
data_folder_path = "../../../datasets/st_data/gold/"
figure_folder_path = f"../../../figures"
benchmark_data_folder_path = "../../../datasets/benchmark_data"
benchmarking_folder_path = "../../../artifacts/single_sample_method_benchmarking"

## 2. Data

### 2.1 Define Data Preparation Function

In [ ]:
def preprocess_data_for_deeplinc(dataset,
                                 cell_type_key):
    # Configure benchmark folder path
    benchmark_data_dataset_folder_path = f"{benchmark_data_folder_path}/{model_name}/{dataset}"
    os.makedirs(benchmark_data_dataset_folder_path, exist_ok=True)
    
    # Load data
    adata = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")

    # Create csv files to run DeepLinc
    counts_df = pd.DataFrame(adata.layers["counts"].toarray(), columns=adata.var_names)
    counts_df.to_csv(f"{benchmark_data_dataset_folder_path}/counts.csv", index=False)

    coords_df = pd.DataFrame(adata.obsm["spatial"], columns=["X", "Y"])
    coords_df.to_csv(f"{benchmark_data_dataset_folder_path}/coords.csv", index=False)

    cell_types_df = pd.DataFrame(adata.obs[cell_type_key])
    cell_types_df.rename(columns={cell_type_key: "Cell_class_name"}, inplace=True)
    cell_types_df["Cell_ID"] = np.arange(len(adata))
    cell_types_df["Cell_class_id"] = cell_types_df["Cell_class_name"].cat.codes
    cell_types_df = cell_types_df[["Cell_ID", "Cell_class_id", "Cell_class_name"]]
    cell_types_df.to_csv(f"{benchmark_data_dataset_folder_path}/cell_types.csv", index=False)
    
    for n_neighbors in [4, 8, 12, 16]:
        # Compute spatial neighborhood graphs
        sq.gr.spatial_neighbors(adata,
                                coord_type="generic",
                                spatial_key="spatial",
                                n_neighs=n_neighbors)
        
        # Make adjacency matrix symmetric
        adj = (adata.obsp["spatial_connectivities"].maximum(
            adata.obsp["spatial_connectivities"].T))
        
        # Write the sparse matrix to a CSV file in dense format
        with open(f"{benchmark_data_dataset_folder_path}/adj{n_neighbors}.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(list(range(adj.shape[0]))) # column headers
            for i in range(adj.shape[0]):
                row_data = np.repeat(0.0, adj.shape[1])
                row_data[adj.nonzero()[1][adj.nonzero()[0] == i]] = 1.0
                row_data = row_data.tolist()
                writer.writerow(row_data)

### 2.2 Preprocess Benchmarking Datasets

In [ ]:
preprocess_data_for_deeplinc(dataset="seqfish_mouse_organogenesis_embryo2",
                             cell_type_key="celltype_mapped_refined")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    preprocess_data_for_deeplinc(dataset=f"seqfish_mouse_organogenesis_subsample_{subsample_pct}pct_embryo2",
                                 cell_type_key="celltype_mapped_refined")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    preprocess_data_for_deeplinc(dataset=f"nanostring_cosmx_human_nsclc_subsample_{subsample_pct}pct_batch5",
                                 cell_type_key="cell_type")

In [ ]:
preprocess_data_for_deeplinc(dataset="slideseqv2_mouse_hippocampus",
                             cell_type_key="cell_type")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    preprocess_data_for_deeplinc(dataset=f"slideseqv2_mouse_hippocampus_subsample_{subsample_pct}pct",
                                 cell_type_key="cell_type")

In [ ]:
preprocess_data_for_deeplinc(dataset="sim1_1105genes_10000locs_strongincrements",
                             cell_type_key="cell_types")

## 3. DeepLinc Model

Now, train DeepLinc models with the ```../DeepLinc/train_deeplinc_models.sh``` script (or rather the slurm job scripts in the same directory that execute this shell script for specific datasets via the sbatch command). The latent space features of trained models will be stored under ```../../../datasets/benchmark_data/deeplinc/{dataset}/runs``` . Delete all ```emb_hidden*_*.npy``` files except for the one of the second hidden layer with the highest number (last epoch where the model improved).

### 3.1 Define Result Retrieval Function

In [ ]:
def extract_number_after_last_letter(s):
    match = re.search(r"[a-zA-Z](\d+)$", s)
    if match:
        return int(match.group(1))
    else:
        return None

In [ ]:
def get_deeplinc_latent(dataset,
                        cell_type_key,
                        niche_type_key=None,
                        adata_new=None):
    # Configure benchmark folder path
    benchmark_data_dataset_folder_path = f"{benchmark_data_folder_path}/{model_name}/{dataset}/runs"
    
    # Configure figure folder path
    dataset_figure_folder_path = f"{figure_folder_path}/{dataset}/method_benchmarking/" \
                                 f"{model_name}/{current_timestamp}"
    os.makedirs(dataset_figure_folder_path, exist_ok=True)
    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:
        adata_original = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        if niche_type_key in adata_original.obs.columns:
            adata_new.obs["niche_type"] = adata_original.obs[niche_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        del(adata_original)

    run_number = 0
    for directory, subdirectory, file_list in os.walk(benchmark_data_dataset_folder_path):
        for file in file_list:
            if "hidden2" in file:
                file_path = os.path.join(directory, file)
                json_path = os.path.join(directory, "run_time.json")

                adata = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
                adata.obsm[latent_key] = np.load(file_path)

                adata_new.obsm[latent_key + f"_run{run_number+1}"] = np.load(file_path)
                
                with open(json_path, "r") as json_file:
                    run_time_dict = json.load(json_file)
                    adata_new.uns[f"{model_name}_model_training_duration_run{run_number+1}"] = run_time_dict["run_time"]

                # Store intermediate adata to disk
                adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad")
                
                run_number += 1

    # Store final adata to disk
    adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad")  

### 3.2 Retrieve Results on Benchmarking Datasets

In [ ]:
get_deeplinc_latent(dataset="seqfish_mouse_organogenesis_embryo2",
                    cell_type_key="celltype_mapped_refined")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    get_deeplinc_latent(dataset=f"seqfish_mouse_organogenesis_subsample_{subsample_pct}pct_embryo2",
                        cell_type_key="celltype_mapped_refined")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    get_deeplinc_latent(dataset=f"nanostring_cosmx_human_nsclc_subsample_{subsample_pct}pct_batch5",
                        cell_type_key="cell_type")

In [ ]:
get_deeplinc_latent(dataset="slideseqv2_mouse_hippocampus",
                    cell_type_key="cell_type")

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    get_deeplinc_latent(dataset=f"slideseqv2_mouse_hippocampus_subsample_{subsample_pct}pct",
                        cell_type_key="cell_type")

In [ ]:
get_deeplinc_latent(dataset="sim1_1105genes_10000locs_strongincrements",
                    cell_type_key="cell_types",
                    niche_type_key="niche_types")